In [20]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [25]:
def yes24DataReader(CategoryNumber, year, month):

    root_url = 'http://www.yes24.com'

    url_1 = 'http://www.yes24.com/24/category/bestseller?CategoryNumber='
    url_2 = '&sumgb=09&year='
    url_3 = '&month='
    url_4 = '&PageNumber='
    url_set = url_1 + CategoryNumber + url_2 + year + url_3 + month + url_4

    book_list=[]

    # 월 별 조회 시 최대 50쪽이지만, 간단한 설명을 위해 2쪽까지만 수집
    for i in range(1, 25):

        url = url_set + str(i)

        res = requests.post(url)
        soup = BeautifulSoup(res.text, 'lxml')
        tag = 'td.goodsTxtInfo > p:nth-child(1) > a:nth-child(1)'
        books = soup.select(tag)

        # 수집 중인 페이지 번호 출력
        print('# Page', i)

        # 개별 도서 정보 수집
        for book in books:

            sub_url = root_url + book.attrs['href']
            sub_res = requests.post(sub_url)
            sub_soup = BeautifulSoup(sub_res.text, 'lxml')

            tag_name = '#yDetailTopWrap > div.topColRgt > div.gd_infoTop > div > h2'
            tag_author = '#yDetailTopWrap > div.topColRgt > div.gd_infoTop > span.gd_pubArea > span.gd_auth > a'
            tag_author2 = '#yDetailTopWrap > div.topColRgt > div.gd_infoTop > span.gd_pubArea > span.gd_auth'
            tag_publisher = '#yDetailTopWrap > div.topColRgt > div.gd_infoTop > span.gd_pubArea > span.gd_pub > a'
         

            tag_listprice = '#yDetailTopWrap > div.topColRgt > div.gd_infoBot > div.gd_infoTbArea > div:nth-child(3) > table > tbody > tr:nth-child(1) > td > span > em'
            tag_listprice2 = '#yDetailTopWrap > div.topColRgt > div.gd_infoBot > div.gd_infoTbArea > div:nth-child(4) > table > tbody > tr:nth-child(1) > td > span > em'
            tag_price = '#yDetailTopWrap > div.topColRgt > div.gd_infoBot > div.gd_infoTbArea > div:nth-child(3) > table > tbody > tr:nth-child(2) > td > span > em'
            tag_price2 = '#yDetailTopWrap > div.topColRgt > div.gd_infoBot > div.gd_infoTbArea > div:nth-child(4) > table > tbody > tr:nth-child(2) > td > span > em'

            tag_page = '#infoset_specific > div.infoSetCont_wrap > div > table > tbody > tr:nth-child(2) > td'
            
            tag_isbn10 = '#infoset_specific > div.infoSetCont_wrap > div > table > tbody > tr:nth-child(4) > td'


            # 기본적인 예외처리를 통한 데이터 수집
            name = sub_soup.select(tag_name)[0].text

            try:
                author = sub_soup.select(tag_author)[0].text
            except:
                author = sub_soup.select(tag_author2)[0].text.strip('\n').strip().replace(' 저','')


            publisher = sub_soup.select(tag_publisher)[0].text
           

            try:
                listprice = sub_soup.select(tag_listprice)[0].text.replace(',','').replace('원','')
            except:
                try:
                    listprice = sub_soup.select(tag_listprice2)[0].text.replace(',','').replace('원','')
                except:
                    listprice = ''

            try:
                price = sub_soup.select(tag_price)[0].text.replace(',','')
            except:
                try:
                    price = sub_soup.select(tag_price2)[0].text.replace(',','')
                except:
                    price = ''

            page = sub_soup.select(tag_page)[0].text
            if '쪽' in page:
                if '확인' in page:
                    page = ''
                else :
                    page = page.split('|')[0].strip().replace('쪽','')
            else :
                page = ''

          


            try :
                isbn10 = sub_soup.select(tag_isbn10)[0].text
                if '확인' in isbn10:
                    isbn10 = ''
                else :
                    isbn10 = sub_soup.select(tag_isbn10)[0].text
            except :
                isbn10 = ''

            book_list.append([name, author, publisher, listprice, price, page, isbn10])

            print('=========>', name)

    # 데이터프레임 컬럼명 지정
    colList = ['name',  'author', 'publisher',  'listprice', 'price', 'page', 'isbn10']

    # 데이터프레임으로 변환
    df = pd.DataFrame(np.array(book_list), columns=colList)

    return df

In [26]:
# 역사 카테고리 번호 : 001001010
CategoryNum='001001014'

# 2019년도
for year in range(2019, 2020):
    print('='*50)
    print('# Year', year)
    print('='*50)

    # 9월
    for month in range(9, 10):
        print('='*50)
        print('# Month', month)
        print('='*50)

        # 월 별 데이터 수집
        df = yes24DataReader(CategoryNum, str(year), str(month))
        
        df

        # 월 별로 수집된 데이터를 CSV 형식 파일로 저장
        df.to_csv(str(year)+'_'+str(month)+'_'+str(CategoryNum)+'.csv', index=False, encoding='CP949')

# Year 2019
# Month 9
# Page 1
=========> 2019 Win-Q 화학분석기사 필기 단기완성
=========> 이상심리학의 기초
=========> 러닝퍼실리테이션
=========> 2019 오직 간호대생을 위한 간호사 면접
=========> 캠벨 생명과학 11판
=========> 일반물리학 1
=========> 정치학의 이해
=========> 상처받은 내면아이 치유
=========> 맨큐의 경제학
=========> 현대 이상심리학
=========> 현대 심리치료와 상담이론
=========> 진보와 빈곤
=========> 시창과 청음연습 1
=========> SPSS 결과표 작성과 해석 방법
=========> 포지셔닝 POSITIONING
=========> 논문 검색과 쓰기 전략
=========> 줌달의 일반화학
=========> 옥스토비의 일반화학
=========> 선택할 자유
=========> 2019 오직 간호대생을 위한 간호사 전공필기
# Page 2
=========> 2019 혼자서 터득하는 세무·회계 사무소 업무 가이드
=========> 감각통합 Q&A
=========> 청소년을 위한 백범일지
=========> 근골격해부학 
=========> 간호사가 말하는 간호사
=========> 2019 10회차 간호조무사 실전문제 총정리
=========> 카레이싱 최후의 비밀
=========> 영어 습득의 이해
=========> 전수환 경영학 핵심요약노트 STEP 4
=========> 주택임대사업자의 모든 것
=========> 사회복지 사례관리론
=========> 뉴만 kinesiology 근육뼈대계통의 기능해부학 및 운동학
=========> 교사다움
=========> 2020 열정샘의 간호조무사 핵심강의+실전 7회 모의고사
=========> 성인간호학 하권
=========> 한국전문소생술 (KALS)
=========> 화성학
=========> 가족치료의 이해
====

=========> 이중나선
=========> 국화와 칼
=========> 여행상품상담실무
=========> 2019 건축물 내진설계기준 및 해설
=========> 프로파일링 이론과 실제
=========> 2019 간호조무사 국가시험문제집
# Page 17
=========> 대상관계 심리치료 실제
=========> NCS 병원 안내 기반 병원 코디네이터
=========> 민법강의
=========> 2019 화재안전기준
=========> 맥머리의 유기화학강의
=========> 15분의 기적
=========> 사회복지행정론
=========> 제제학
=========> 거시경제론
=========> 유아음악교육
=========> 원가관리회계
=========> 집단상담 이론과 실제
=========> 보육실습실무
=========> 고급 영문법해설
=========> 기초부터 시작하는 PLC : 멜섹Q
=========> 특수교육공학
=========> 현대인간공학
=========> 진보와 빈곤
=========> 2015 한국표준질병 사인분류
=========> 독서교육론
# Page 18
=========> Nichols의 가족치료 이론과 실제
=========> 강박증 극복하기 
=========> 부모상담 실제
=========> 게슈탈트 심리치료
=========> 똑똑한 인지행동치료  
=========> 1001가지 해결중심 질문들
=========> 근육해부학 
=========> 근육뼈대계의 기능과 운동해부학 
=========> 유아교육기관에서의 부모 교육과 지원
=========> 서울대 한국어 2A Student's Book with CD-ROM
=========> 초보자를 위한 학위논문 작성법
=========> 범죄의 해부학
=========> 스피치 커뮤니케이션
=========> 미술치료학
=========> 진정한 사람되기
=========> 기초정수론
=========> 상담심리학의 기초
=========>

In [15]:
url = 'http://www.yes24.com/24/category/bestseller?CategoryNumber=001001010&sumgb=09&year=2022&month=12&PageNumber=1'
res = requests.post(url)
soup = BeautifulSoup(res.text, 'lxml')
tag = 'td.goodsTxtInfo > p:nth-child(1) > a:nth-child(1)'
books = soup.select(tag)
print(books)

[<a href="/Product/Goods/114945286">나의 문화유산답사기 11 : 서울편 3  </a>, <a href="/Product/Goods/115097094">창발의 시대</a>, <a href="/Product/Goods/114945302">나의 문화유산답사기 12 : 서울편 4 </a>, <a href="/Product/Goods/113477617">눈에 보이지 않는 지도책</a>, <a href="/Product/Goods/1940233">총, 균, 쇠</a>, <a href="/Product/Goods/115027357">나의 문화유산답사기 서울편 세트 </a>, <a href="/Product/Goods/104409067">거꾸로 읽는 세계사</a>, <a href="/Product/Goods/115381226">2023 황현필의 한국사 일력</a>, <a href="/Product/Goods/30217950">지리의 힘</a>, <a href="/Product/Goods/114272631">더 넓은 세계사</a>, <a href="/Product/Goods/108422196">지리의 힘 2</a>, <a href="/Product/Goods/110243751">유럽 도시 기행 2</a>, <a href="/Product/Goods/114295799">벌거벗은 한국사 : 사건편 </a>, <a href="/Product/Goods/112032745">벌거벗은 한국사 : 인물편 </a>, <a href="/Product/Goods/74360497">역사의 쓸모</a>, <a href="/Product/Goods/44263948">나의 문화유산답사기 9 : 서울편 1</a>, <a href="/Product/Goods/75447737">유럽 도시 기행 1</a>, <a href="/Product/Goods/112890411">전쟁과 약, 기나긴 악연의 역사</a>, <a href="/Product/Goods/8500082">총, 균, 

In [ ]:
http://www.yes24.com/24/category/bestseller?CategoryNumber=001001010&sumgb=09&year=2022&month=12&PageNumber=1